# Embedding and Document Clustering

In [2]:
import os
import numpy as np
import pandas as pd
import sys
sys.path.append(os.path.abspath(".."))

import plotly.plotly as py
from sklearn import metrics
from hdbscan import HDBSCAN
from plotly.graph_objs import *
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

from url2vec.util.plotter import *
from url2vec.util.metrics import *
from url2vec.util.seqmanager import *

from sklearn.metrics import pairwise_distances
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

The crawling proccess has been done in two different ways:

- **No costraint**: the crawler follows a random outlink from all of the outlinks in a given page
- **List costraint**: the crawler follows a random outlink but only from the outlinks in "lists"

Here we're loading all the files that the crawler has generated to train word2vec model.

See the [Dataset README](https://github.com/chrisPiemonte/url2vec/tree/master/dataset "Dataset") for further information.


In [3]:
"""No-Costraint"""
nocostraint_path   = os.getcwd() + "/../dataset/cs.illinois.edu_NoConstraint.words1000.depth10/"
vertex_nc_path     = nocostraint_path + "vertex.txt"
map_nc_path        = nocostraint_path + "urlsMap.txt"

# code -> content_string - dict 
content_nc_map = get_content_map(vertex_nc_path)
# code -> longurl - dict 
url_nc_map = get_urlmap(map_nc_path)
# document no-costraint list
pages_content_nc = [content_nc_map[key] for key in content_nc_map]
# codes no-costraint list
codes_nc = [key for key in content_nc_map]
# urls no-costraint list
urls_nc = [url_nc_map[key] for key in content_nc_map]

In [4]:
"""List-Costraint"""
listcostraint_path = os.getcwd() + "/../dataset/cs.illinois.edu_ListConstraint.words1000.depth10/"
vertex_lc_path     = listcostraint_path + "vertex.txt"
map_lc_path        = listcostraint_path + "urlsMap.txt"

# code -> content_string - dict 
content_lc_map = get_content_map(vertex_lc_path)
# code -> longurl - dict 
url_lc_map = get_urlmap(map_lc_path)
# document list-costraint list
pages_content_lc = [content_lc_map[key] for key in content_lc_map]
# codes list-costraint list
codes_lc = [key for key in content_lc_map]
# urls list-costraint list
urls_lc = [url_lc_map[key] for key in content_lc_map]

### TF-IDF matrix

Here is defined **term frequency - inverse document frequency** (tf-idf) vectorizer parameters and then convert the documents (web pages) list into a tf-idf matrix.

To get a Tf-idf matrix, first count word occurrences by document. This is transformed into a **document-term matrix** (dtm).![Alt text](http://www.codeproject.com/KB/WPF/NNMFSearchResultClusterin/table.jpg "Very nice")

This is also just called a term frequency matrix.
Then apply the term frequency-inverse document frequency weighting: words that occur frequently within a document but not frequently within the corpus receive a higher weighting as these words are assumed to contain more meaning in relation to the document.

A couple things to note about the parameters defined below:

**max_df**: this is the maximum frequency within the documents a given feature can have to be used in the tfi-idf matrix. If the term is in greater than 80% of the documents it probably cares little meanining

**min_idf**: this could be an integer (e.g. 5) and the term would have to be in at least 5 of the documents to be considered. Here I pass 0.1; the term must be in at least 10% of the document.

**ngram_range**: this just means I'll look at unigrams, bigrams and trigrams.

In [5]:
# TFIDF vectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_df = 0.8,
    max_features = 200000,
    min_df = 0.1,
    stop_words = 'english',
    use_idf = True,
    tokenizer = tokenize_and_stem,
    ngram_range = (1,3)
)

"""TFIDF matrix No-Costraint"""
tfidf_matrix_nc = tfidf_vectorizer.fit_transform(pages_content_nc)

"""TFIDF matrix List-Costraint"""
tfidf_matrix_lc = tfidf_vectorizer.fit_transform(pages_content_lc)

### Word2Vec
Word embedding algorithm. Word2vec is a two-layer neural net that processes text. Its input is a text corpus and its output is a set of vectors: feature vectors for words in that corpus.

Here we're apply word2vec (skip-gram with negative sampling) to sequences generated by crawling the web site.

In [6]:
"""No-Costraint"""
sequences_nc = nocostraint_path + "sequenceIDs.txt"

# because of the damn generators
vocab_sequences_nc = get_sequences(sequences_nc)
train_sequences_nc = get_sequences(sequences_nc)

w2v_model_nc = Word2Vec(min_count=1, negative=5, size=100)
w2v_model_nc.build_vocab(vocab_sequences_nc)
w2v_model_nc.train(train_sequences_nc)

w2v_vecs_nc = np.array([w2v_model_nc[key] for key in content_nc_map])

"""List-Costraint"""
sequences_lc = listcostraint_path + "sequenceIDs.txt"

# because of the damn generators
vocab_sequences_lc = get_sequences(sequences_lc)
train_sequences_lc = get_sequences(sequences_lc)

w2v_model_lc = Word2Vec(min_count=1, negative=5, size=100)
w2v_model_lc.build_vocab(vocab_sequences_lc)
w2v_model_lc.train(train_sequences_lc)

w2v_vecs_lc = np.array([w2v_model_lc[key] for key in content_lc_map])

### Latent Semantic Analysis (LSA)
Dimensionality reduction using **truncated SVD** (aka LSA).
This transformer performs linear dimensionality reduction by means of truncated singular value decomposition (SVD). 

It is very similar to PCA, but operates on sample vectors directly, instead of on a covariance matrix.

In particular, truncated SVD works on term count/tf-idf matrices as returned by the vectorizers. In that context, it is known as latent semantic analysis (LSA).

In [7]:
svd = TruncatedSVD(n_components=50, algorithm="arpack", random_state=1)

"""No-Costraint"""
pages_tfidf_vecs_nc = svd.fit_transform(tfidf_matrix_nc)

"""List-Costraint"""
pages_tfidf_vecs_lc = svd.fit_transform(tfidf_matrix_lc)

### Combining Vectors from word2vec and tf-idf
Appending tf-idf vector at the end of the relevant word2vec vector for each page.

In [8]:
"""No-Costraint"""
combined_vecs_nc = [np.append(w2v_vecs_nc[i], pages_tfidf_vecs_nc[i]) 
                    for i in range(len(pages_tfidf_vecs_nc))]

"""List-Costraint"""
combined_vecs_lc = [np.append(w2v_vecs_lc[i], pages_tfidf_vecs_lc[i]) 
                    for i in range(len(pages_tfidf_vecs_lc))]

### T-SNE
Applying t-SNE for dimensionality reduction. We need two dimensional vectors for visualization purposes.

In [9]:
tsne = TSNE(n_components=2, random_state=1)

"""No-Costraint"""
twodim_nc = tsne.fit_transform(combined_vecs_nc)

"""List-Costraint"""
twodim_lc = tsne.fit_transform(combined_vecs_lc)

### K-Means Clustering

K-means initializes with a pre-determined number of clusters. Each observation is assigned to a cluster (cluster assignment) so as to minimize the within cluster sum of squares. Next, the mean of the clustered observations is calculated and used as the new cluster centroid. Then, observations are reassigned to clusters and centroids recalculated in an iterative process until the algorithm reaches convergence.

In [18]:
kmeans = KMeans(n_clusters=15)

"""No-Costraint"""
kmeans.fit(combined_vecs_nc)
kmeans_labels_nc = kmeans.labels_
kmeans_colors_nc = [get_color(i) for i in kmeans_labels_nc]

"""List-Costraint"""
#kmeans.fit(tfidf_matrix_lc)
kmeans.fit(combined_vecs_lc)
kmeans_labels_lc = kmeans.labels_
kmeans_colors_lc = [get_color(i) for i in kmeans_labels_lc]

#### K-Means no-costraint plot

In [19]:
kmeans_data_nc = scatter_plot(twodim_nc, word_labels=urls_nc, colors=kmeans_colors_nc)
py.iplot(kmeans_data_nc, filename="K-Means TFIDF-W2V Clustering - No-Costraint")

<div>
    <a href="https://plot.ly/~chrispolo/42" 
        target="_blank" title="y" 
        style="display: block; text-align: center;">
            <img src="../dataset/img/nc_ed_wordvectors_scatter_plot_KMEANS.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:42"  src="https://plot.ly/embed.js" async></script>
</div>

#### K-Means list-costraint plot

In [21]:
kmeans_data_lc = scatter_plot(twodim_lc, word_labels=urls_lc, colors=kmeans_colors_lc)
py.iplot(kmeans_data_lc, filename="K-Means TFIDF-W2V Clustering - List-Costraint")

<div>
    <a href="https://plot.ly/~chrispolo/36" 
        target="_blank" title="y" 
        style="display: block; text-align: center;">
            <img src="../dataset/img/lc_ed_wordvectors_scatter_plot_KMEANS.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:36"  src="https://plot.ly/embed.js" async></script>
</div>

### HDBSCAN Clsustering

Hierarchical Density-Based Spatial Clustering of Applications with Noise. Performs DBSCAN over varying epsilon values and integrates the result to find a clustering that gives the best stability over epsilon. This allows HDBSCAN to find clusters of varying densities (unlike DBSCAN), and be more robust to parameter selection.

**params**:

- **min_cluster_size** : minimum nodes to form a cluster

In [13]:
hdbscan = HDBSCAN(min_cluster_size=7)

"""No-Costraint"""
hdbscan_labels_nc = hdbscan.fit_predict(combined_vecs_nc)
hdbscan_colors_nc = [get_color(n_clust) for n_clust in hdbscan_labels_nc]

print "Clusters found with HDBSCAN on No-costraint Dataset:", len(set(hdbscan_labels_nc))
print [label for label in set(hdbscan_labels_nc)], "\n"

"""List-Costraint"""
hdbscan_labels_lc = hdbscan.fit_predict(combined_vecs_lc)
hdbscan_colors_lc = [get_color(n_clust) for n_clust in hdbscan_labels_lc]

print "Clusters found with HDBSCAN on List-costraint Dataset:", len(set(hdbscan_labels_nc))
print [label for label in set(hdbscan_labels_nc)]

Clusters found with HDBSCAN on No-costraint Dataset: 12
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, -1] 

Clusters found with HDBSCAN on List-costraint Dataset: 12
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, -1]


#### HDBSCAN no-costraint plot

In [22]:
hdbscan_data_nc = scatter_plot(twodim_nc, word_labels=urls_nc, colors=hdbscan_colors_nc)
py.iplot(hdbscan_data_nc, filename="HDBSCAN TFIDF-W2V Clustering - No-Costraint")

<div>
    <a href="https://plot.ly/~chrispolo/38" 
        target="_blank" title="y" 
        style="display: block; text-align: center;">
            <img src="../dataset/img/nc_ed_wordvectors_scatter_plot_HDBSCAN.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:38"  src="https://plot.ly/embed.js" async></script>
</div>

#### HDBSCAN list-costraint plot

In [23]:
hdbscan_data_lc = scatter_plot(twodim_lc, word_labels=urls_lc, colors=hdbscan_colors_lc)
py.iplot(hdbscan_data_lc, filename="HDBSCAN TFIDF-W2V Clustering - List-Costraint")

<div>
    <a href="https://plot.ly/~chrispolo/40" 
        target="_blank" title="y" 
        style="display: block; text-align: center;">
            <img src="../dataset/img/lc_ed_wordvectors_scatter_plot_HDBSCAN.png" 
                alt="y" style="max-width: 100%;width: 1121px;"  
                width="100%" onerror="this.onerror=null;this.src='https://plot.ly/404';" />
    </a>
    <script data-plotly="chrispolo:40"  src="https://plot.ly/embed.js" async></script>
</div>

## Evaluation
Evaluating the performance of a clustering algorithm is not as trivial as counting the number of errors or the precision and recall of a supervised classification algorithm. In particular any evaluation metric should not take the absolute values of the cluster labels into account but rather if this clustering define separations of the data similar to some ground truth set of classes or satisfying some assumption such that members belong to the same class are more similar that members of different classes according to some similarity metric.

See the [scikit-learn documentaion](http://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation "ti") for futher information

### Metrics:

- **Homogeneity**: each cluster contains only members of a single class


- **Completeness**: all members of a given class are assigned to the same cluster


- **Adjusted Rand index**: Given the knowledge of the *ground truth* class assignments and our clustering algorithm assignments of the same samples, the adjusted Rand index is a function that measures the similarity of the two assignments, ignoring permutations and with chance normalization


- **V-measure**: The V-measure is actually equivalent to the mutual information (NMI) discussed above normalized by the sum of the label entropies


- **Mutual Information based scores**: Given the knowledge of the ground truth class assignments and our clustering algorithm assignments of the same samples, the Mutual Information is a function that measures the agreement of the two assignments, ignoring permutations. Two different normalized versions of this measure are available, Normalized Mutual Information(NMI) and Adjusted Mutual Information(AMI). NMI is often used in the literature while AMI was proposed more recently and is normalized against chance


- **Silhouette**: If the ground truth labels are not known, evaluation must be performed using the model itself. The Silhouette Coefficient is an example of such an evaluation, where a higher Silhouette Coefficient score relates to a model with better defined clusters. The score is bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters. The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.

In [16]:
gt = GroundTruth(os.getcwd() + "/../dataset/ground_truth/urlToMembership.txt")

ground_truth_nc = [int(gt.get_groundtruth(url_nc_map[key])) for key in content_nc_map]
ground_truth_lc = [int(gt.get_groundtruth(url_lc_map[key])) for key in content_lc_map]

pd.DataFrame(get_confusion_table(ground_truth_lc, [int(x) for x in kmeans_labels_lc]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0,0,0,0,0,0,0,18,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0
2,0,0,0,92,1,0,0,0,0,0,0,0,0,76,0
3,0,0,0,0,0,28,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
6,40,177,0,0,27,0,0,0,0,0,0,0,0,0,47
7,1,0,0,0,22,0,28,0,0,263,0,0,0,2,0
8,1,2,0,0,7,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,57,0,0,0,0


In [17]:
metrics_df = pd.DataFrame([
        [
            # hdbscan nocostraint
            metrics.homogeneity_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.completeness_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.v_measure_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.adjusted_rand_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.adjusted_mutual_info_score(ground_truth_nc, hdbscan_labels_nc),
            metrics.silhouette_score(np.array(combined_vecs_nc), hdbscan_labels_nc, metric='euclidean')
        ],
        [
            # kmeans nocostraint
            metrics.homogeneity_score(ground_truth_nc, kmeans_labels_nc),
            metrics.completeness_score(ground_truth_nc, kmeans_labels_nc),
            metrics.v_measure_score(ground_truth_nc, kmeans_labels_nc),
            metrics.adjusted_rand_score(ground_truth_nc, kmeans_labels_nc),
            metrics.adjusted_mutual_info_score(ground_truth_nc, kmeans_labels_nc),
            metrics.silhouette_score(np.array(combined_vecs_nc), kmeans_labels_nc, metric='euclidean')
        ],
        [
            # hdbscan listcostraint
            metrics.homogeneity_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.completeness_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.v_measure_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.adjusted_rand_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.adjusted_mutual_info_score(ground_truth_lc, hdbscan_labels_lc),
            metrics.silhouette_score(np.array(combined_vecs_lc), hdbscan_labels_lc, metric='euclidean')
        ],
        [
            # kmeans listcostraint
            metrics.homogeneity_score(ground_truth_lc, kmeans_labels_lc),
            metrics.completeness_score(ground_truth_lc, kmeans_labels_lc),
            metrics.v_measure_score(ground_truth_lc, kmeans_labels_lc),
            metrics.adjusted_rand_score(ground_truth_lc, kmeans_labels_lc),
            metrics.adjusted_mutual_info_score(ground_truth_lc, kmeans_labels_lc),
            metrics.silhouette_score(np.array(combined_vecs_lc), kmeans_labels_lc, metric='euclidean')
        ]],
        index=[ 
            "NoCostraint - HDBSCAN", 
            "NoCostraint - K-MEANS",  
            "ListCostraint - HDBSCAN", 
            "ListCostraint - K-MEANS"
        ],
        columns=[
            "Homogeneity", 
            "Completeness", 
            "V-Measure core", 
            "Adjusted Rand index", 
            "Mutual Information",
            "Silhouette"
        ])

metrics_df

,Homogeneity,Completeness,V-Measure core,Adjusted Rand index,Mutual Information,Silhouette
NoCostraint - HDBSCAN,0.732700,0.753440,0.742925,0.720437,0.718552,0.207000
NoCostraint - K-MEANS,0.881165,0.806894,0.842396,0.829892,0.794896,0.319759
ListCostraint - HDBSCAN,0.654103,0.612894,0.632828,0.324909,0.599170,0.120338
ListCostraint - K-MEANS,0.854799,0.688500,0.762690,0.648750,0.677258,0.257286
